In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
os.chdir('./gdrive/MyDrive/RNN_for_GCPL')

In [ ]:
import os
os.chdir('../')

In [ ]:
import os
import sys
import numpy as np
from sys import getsizeof
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from itertools import product
import pickle

import math
import shutil
import torch
from torch.utils.data import Dataset
import scipy as sp
from scipy.optimize import curve_fit
import sklearn
from Code.setup import *
import datetime as dt
import time
import torch.nn as nn
import copy
from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
import RNN_for_GCPL.setup

In [ ]:
os.chdir('../RNN_for_GCPL/')

In [ ]:

savepath = r'data/v4/120/main'
dataset_rs2 = GCPL_dataset_resampled3(savepath)

In [ ]:
seedEverything(seed=DEFAULT_RANDOM_SEED)
input_dim = 2
output_dim = 20
num_layers = 2
bidir= True
lr = 5e-4
init_model = MyGRU(input_dim, output_dim, num_layers=num_layers,bidir = bidir)
criterion = nn.MSELoss(reduction='none')
best_model = MyGRU(input_dim, output_dim, num_layers=num_layers,bidir = bidir)
# kf = KFold(5, shuffle=True, random_state=DEFAULT_RANDOM_SEED)
gkf = GroupKFold(4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
models = []
# params = [ (30,3,False), (60, 4, True)]
params = [(20, 1, True), (20, 2, True), (30, 3, True)]
for a,b,c in params:
    model = MyGRU(input_dim, a, num_layers=b, bidir=c)
    models.append(model)
sampling = [60, 120, 180]

In [ ]:
probs = balancing(dataset_rs2, 10)
soh, info = statistics(dataset_rs2)
clear_output()

In [ ]:
draw_every = 25
num_epochs = 75
save_every = 10
batch_size = 256
for m in sampling:
    dataset_rs2 = GCPL_dataset_resampled3(f'data/v4/{m}/main')
    dataset_validate = GCPL_dataset_resampled3(f'data/v4/{m}/validate')
    for l, init_model in enumerate(models):
        path = f'./models/v10/{m}/{l}'
        for k, (train_indices, val_indices) in enumerate(gkf.split(dataset_rs2, groups=info.Pouch)):
            seedEverything(seed=DEFAULT_RANDOM_SEED)
            train_set = torch.utils.data.Subset(dataset_rs2, train_indices)
            val_set = torch.utils.data.Subset(dataset_rs2, val_indices)
            sampler = torch.utils.data.WeightedRandomSampler(probs[train_indices], len(train_indices)) 
            train_loader = torch.utils.data.DataLoader(train_set, batch_size, collate_fn=collate_batch, sampler=sampler)
            val_loader = torch.utils.data.DataLoader(val_set, batch_size, shuffle=True, collate_fn=collate_batch)
            model = copy.deepcopy(init_model)
            model.to(device)
            optimizer =  torch.optim.Adam(model.parameters(), lr=lr)
            sheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)
            handler = ModelHandler(model, 1e6, f'20 neurons, 2 layers, 120s sampling, 80% train, 20% val, k-fold No {k}.', path= path, kfold=True)
            pp = ProgressPlotter()
            counter = 0
            for i in range(num_epochs):
                model.train()
                loss_val = 0
                loss_train = 0
                for data, labels in train_loader:
                    optimizer.zero_grad()
                    preds = model(data.to(device))
                    loss = torch.sum(criterion(preds, labels.to(device)))
                    loss.backward()
                    optimizer.step()
                    loss_train += loss.detach().cpu()
                
                
                with torch.no_grad():
                    loss_train = loss_train/len(train_loader)
                    pp.add_scalar('loss_train', loss_train.detach().cpu().numpy())
                    model.eval()
                    for data, labels in val_loader:
                        preds = model(data.to(device))
                        loss = torch.sum(criterion(preds, labels.to(device)))
                        loss_val += loss.detach().cpu()
                    loss_val = loss_val/len(val_set)
                    sheduler.step(loss_val)
                    handler.check_loss(loss_val, draw_every)
                    pp.add_scalar('loss_val', loss_val.detach().cpu().numpy())
                    if loss_val > handler.best_loss:
                        counter +=1
                    else:
                        counter = 0
                    if (i+1)%draw_every == 0:
                        pp.display([['loss_train', 'loss_val']])
                    print(i, counter, loss_train*2500, loss_val*2500)
                if (i+1)%save_every==0:
                    handler.save(kfold_number=k)
                    
                    
            handler.add_pp(pp)
            handler.display()
            handler.save(kfold_number=k)
            with torch.no_grad():
                loss_val = []
                for data, labels in val_loader:
                    preds = model(data.to(device))
                    loss = criterion(preds, labels.to(device))
                    loss_val.append(loss)
                loss_val = torch.hstack(loss_val)
                print(loss_val.mean(), loss_val.std())


In [ ]:
loss = []
train_set = []
val_set = []
model = copy.deepcopy(init_model)
optimizer =  torch.optim.Adam(model.parameters(), lr=lr) 
path= f'./models/v127/2'
handler = ModelHandler(model, optimizer, train_set, train_set, 1e6, f'one batch, model No k-fold No .', path= f'./models/v2/', kfold=True)
batches = next(os.walk(path))[1]
loss_ = []
epochs_ = []
for batch in batches:
    batchpath = path+ '/' + batch
    handler.load(batchpath)
    loss_.append(handler.best_loss.cpu())
loss.append(loss_)
loss = np.array(loss)*2500
print(str(np.mean(loss))+ '±' + str(np.std(loss)))
loss



In [ ]:
def load(path_):
    if path_.split('.')[-1] == 'pt':
        path = '/'.join(path.split('/')[:-1])
        name = path_.split('/')[-1]
        checkpoint = torch.load(path+ '/' + name, map_location=torch.device('cpu'))
        loss = [checkpoint['loss'].numpy()]
    else:
        path = path_
        files = next(os.walk(path_))[2]
        x = lambda x: int(x.split('_')[-1].split('.')[0].split('(')[0])
        files.sort(key=x)
        loss = []
        for name in files:
            checkpoint = torch.load(path+ '/' + name, map_location=torch.device('cpu'))
            loss.append(checkpoint['loss'].numpy())
    return np.array(loss)


In [ ]:
path = f'./models/v10'
loss = {}
for root, dirs, files in os.walk(path):
    if dirs:
        continue
    if not files:
        continue

    loss[root] = load(root)*2500
loss

In [ ]:
path = f'./models/v10/120/0/0'
model = copy.deepcopy(models[0])
handler = ModelHandler(model)
handler.load(path)

In [ ]:
def collate_batch_named(batch, named=True):
    sample_list = []
    label_list = []
    pouch = []
    cycle = []
    for i in batch:
        sample = np.stack([i['E'], i['I']],axis=-1)
        sample_list.append(torch.tensor(sample, dtype=torch.float32))
        label_list.append(i['SoH'])
        pouch.append(i['Pouch'])
        cycle.append(i['Cycle'])
    sequence_pad = nn.utils.rnn.pad_sequence(sample_list)
    labels = torch.tensor(label_list, dtype=torch.float32)
    if not named:
        return sequence_pad, labels    
    else:
        return sequence_pad, labels, pouch, cycle

In [ ]:
dataset_validate = GCPL_dataset_resampled3(f'data/v4/120/validate')


In [ ]:
batch_size=256
val_loader = torch.utils.data.DataLoader(dataset_validate, batch_size, shuffle=True, collate_fn=collate_batch_named)
with torch.no_grad():
    loss = []
    pouches = []
    soh = []
    pred = []
    cycles = []
    for data, labels, pouch, cycle in val_loader:
        preds = handler.best_model(data)
        pred.extend(preds)
        loss_ = criterion(preds, labels)
        loss.append(loss_)
        pouches.extend(pouch)
        cycles.extend(cycle)
        soh.extend(labels)
    loss = torch.hstack(loss)
    soh = torch.Tensor(soh)*50+50
    pred = torch.Tensor(pred)*50+50
    loss *= 2500 
    results = pd.DataFrame({'Pouch':pouches,'Cycle':cycles, 'Loss':loss, 'SoH':soh,'Pred':pred} )
    results.sort_values(by='Loss',axis=0, ascending=0,inplace=True)
    print(results.Loss.mean())

In [ ]:
dataset_rs2[900]

In [ ]:
results[:50]

In [ ]:
loss = {}
for s in sampling:
    l_ = {}
    for i in range(3):
        path = f'./models/v10/{s}/{i}'
        batches = next(os.walk(path))[1]
        loss_ = []
        for batch in batches:
            batchpath = path+ '/' + batch
            a = load(batchpath)
            loss_.append(a)
        loss_ = torch.tensor(loss_)*2500
        l_[i] = loss_
        print(s, i, f'{loss_.mean():.3}±{loss_.std():.3}')
    loss[s] = l_

In [ ]:
length = []
soh = []
info = pd.DataFrame({
            'Pouch':[],
            'SoH':[],
            "Len":[],
            })
for i in range(len(dataset_rs2)):
    one = len(dataset_rs2[i]['E'])
    soh = dataset_rs2[i]['SoH']*50+50
    if np.any(np.isnan(dataset_rs2[i]['E'])):
        print(i)
    pouch = dataset_rs2[i]['Pouch']
    if i%1000 == 0:
        print(i)
    info.loc[len(info)] = [pouch, soh, one]
info